# Book market research

*This is an assignment given as part of the graduation project for Yandex100 program.  

This is not a whole project but only an example of some SQL queries I have written.*

**This is a study of book services apps. It will be used to prepare a value proposition for a new app for book lovers.  
The database contains data on books, publishers, authors, and customer ratings and reviews of books.**

In [4]:
# ! pip install PyMySQL

In [5]:
# import libraries
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import pymysql

In [6]:
#connect to database
db_config = {'user':'some_user',         # user name
             'pwd': 'somepassword', # password
             'host':'some.local.host.net',
             'port':1234,              # connection port
             'db': 'books-db'} # the name of the data base

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [7]:
connection = engine.connect()

### Study the data

In [8]:
engine.table_names()

['orders',
 'visits',
 'advertisment_costs',
 'publishers',
 'authors',
 'reviews',
 'ratings',
 'books']

In [9]:
books = f'''SELECT *
FROM books'''

In [10]:
books_load = pd.read_sql(books,engine)

display(books_load.head())
books_load.info()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
book_id             1000 non-null int64
author_id           1000 non-null int64
title               1000 non-null object
num_pages           1000 non-null int64
publication_date    1000 non-null object
publisher_id        1000 non-null int64
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


In [11]:
authors = f'''
SELECT *
FROM authors'''

In [12]:
authors_load = pd.read_sql(authors, engine)

display(authors_load.head())
authors_load.info()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
author_id    636 non-null int64
author       636 non-null object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


In [13]:
publishers = f'''
SELECT *
FROM publishers'''

In [14]:
publishers_load = pd.read_sql(publishers, engine)

display(publishers_load.head())
publishers_load.info()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
publisher_id    340 non-null int64
publisher       340 non-null object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


In [15]:
reviews = f'''
SELECT *
FROM reviews'''

In [16]:
reviews_load = pd.read_sql(reviews, engine)

display(reviews_load.head())
reviews_load.info()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
review_id    2793 non-null int64
book_id      2793 non-null int64
username     2793 non-null object
text         2793 non-null object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


In [17]:
ratings = f'''
SELECT *
FROM ratings'''

In [18]:
ratings_load = pd.read_sql(ratings, engine)

display(ratings_load.head())
ratings_load.info()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
rating_id    6456 non-null int64
book_id      6456 non-null int64
username     6456 non-null object
rating       6456 non-null int64
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


### SQL queries

**Find the number of books released after January 1, 2000**

In [19]:
query_one = f'''
SELECT COUNT(book_id) AS book_count
FROM books
WHERE CAST (publication_date AS date) > '2000-01-01';
'''

In [20]:
new_books = pd.read_sql(query_one,engine)
new_books

,book_count
0,819


Out of 1,000 books in the database 819 were published after January 1, 2000.  
This database mostly consists of relatively new books, not classics.   

**Find the number of user reviews and the average rating for each book**

In [21]:
query_two_1 = f'''
SELECT 
    DISTINCT books.book_id AS book_id,
    books.title AS title,
    COUNT(DISTINCT reviews.review_id) AS review_count,
    AVG(ratings.rating) AS avg_rating 
FROM
    books
INNER JOIN reviews ON reviews.book_id = books.book_id
INNER JOIN ratings ON ratings.book_id = books.book_id
GROUP BY 
    books.book_id
ORDER BY review_count DESC
LIMIT 20;

'''

In [22]:
reviews_ratings_1 = pd.read_sql(query_two_1,engine)
reviews_ratings_1

,book_id,title,review_count,avg_rating
0,948,Twilight (Twilight #1),7,3.662500
1,207,Eat Pray Love,6,3.395833
2,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,497,Outlander (Outlander #1),6,4.125000
5,627,The Alchemist,6,3.789474
6,656,The Book Thief,6,4.264151
7,673,The Catcher in the Rye,6,3.825581
8,695,The Curious Incident of the Dog in the Night-Time,6,4.081081
9,696,The Da Vinci Code (Robert Langdon #2),6,3.830508


In [23]:
query_two_2 = f'''
SELECT 
    DISTINCT books.book_id AS book_id,
    books.title AS title,
    COUNT(DISTINCT reviews.review_id) AS review_count,
    AVG(ratings.rating) AS avg_rating 
FROM
    books
INNER JOIN reviews ON reviews.book_id = books.book_id
INNER JOIN ratings ON ratings.book_id = books.book_id
GROUP BY 
    books.book_id
ORDER BY avg_rating DESC
LIMIT 50;

'''

In [24]:
reviews_ratings_2 = pd.read_sql(query_two_2,engine)
reviews_ratings_2

,book_id,title,review_count,avg_rating
0,17,A Dirty Job (Grim Reaper #1),4,5.000000
1,20,A Fistful of Charms (The Hollows #4),2,5.000000
2,55,A Woman of Substance (Emma Harte Saga #1),2,5.000000
3,57,Act of Treason (Mitch Rapp #9),2,5.000000
4,62,Alas Babylon,2,5.000000
5,76,Angels Fall,2,5.000000
6,86,Arrows of the Queen (Heralds of Valdemar #1),2,5.000000
7,136,Captivating: Unveiling the Mystery of a Woman'...,2,5.000000
8,169,Crucial Conversations: Tools for Talking When ...,2,5.000000
9,182,Dead Souls,2,5.000000


We can see that books with 5.0 rating have relatively low amount of reviews. The most reviewed book has 7 reviews.  
We can conclude that the more reviews a book gets, the more chances that it gets a lower rating. This makes sense- the more opinions there are, the more diverse those opinions are. 

**Identify the publisher that has released the greatest number of books with more than 50 pages (this will help you exclude brochures and similar publications from your analysis)**

In [25]:
query_three = f'''
SELECT 
    publishers.publisher AS publisher,
    COUNT(books.book_id)
FROM
    books
INNER JOIN publishers ON books.publisher_id = publishers.publisher_id
WHERE books.num_pages > 50
GROUP BY publishers.publisher
ORDER BY count DESC
LIMIT 1;
'''

In [26]:
biggest_publisher = pd.read_sql(query_three,engine)
biggest_publisher

,publisher,count
0,Penguin Books,42


Penguin Books is the largest publisher here. 

**Identify the author with the highest average book rating (look only at books with at least 50 ratings)**

In [27]:
query_four = f'''
SELECT 
    authors.author AS author_name,
    AVG(SUBQ.avg_rating) AS avg_rating
FROM
    (SELECT DISTINCT ratings.book_id,
            AVG(ratings.rating) AS avg_rating,
            COUNT(ratings.rating_id) AS cnt_ratings
     FROM ratings
     GROUP BY ratings.book_id) AS SUBQ
INNER JOIN books ON SUBQ.book_id = books.book_id
INNER JOIN authors ON books.author_id = authors.author_id
WHERE SUBQ.cnt_ratings >= 50
GROUP BY authors.author
ORDER BY avg_rating DESC
LIMIT 1;
'''

In [28]:
most_rated_author = pd.read_sql(query_four, engine)
most_rated_author

,author_name,avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844


The highest rated author in J.K. Rowling (and the illustrator of the U.S edition of Harry Potter books). No surprise here :)   

**Find the average number of text reviews among users who rated more than 50 books**

In [29]:
query_five = f'''
SELECT
    AVG(SUBQ.cnt_reviews) AS average_number
FROM
    (SELECT  
        COUNT(DISTINCT reviews.review_id) AS cnt_reviews  
        FROM reviews
        LEFT JOIN ratings ON ratings.username = reviews.username
        GROUP BY reviews.username
        HAVING COUNT(DISTINCT ratings.rating_id) > 50) AS SUBQ
        ;    
'''

In [30]:
top_reviewer = pd.read_sql(query_five, engine)
top_reviewer

,average_number
0,24.333333


The average number of text reviews for the top rating users is 24.3 reviews. 